SIDE NOTE: would love to get any critiques, hints, tips, oberservations on any level not just on ML (but obviously ML as the priority).  Beginner Python learner (though I took the Full Stack Course) so any suggestions there are welcome.  Thanks.

Will need to import these libs plus setting some global variables

In [1]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
import matplotlib.cm as cm
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Might need to change the path of the included libraries.
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches

# Variables
round_number = 27 # for MLS only
target_col = 'points'
ignore_cols = ['games_played', 'match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled']
models = ['log', 'svc', 'gmm', 'knn', 'gnb']

""" this variable 'testing' should be false if using CSV's and not pulling from the database. """
testing = False

INITIALIZED...


Pulling in the data either from the Database or the CSV (CSV for testers).  Data has some added features in between the database and 'raw_data'.  Essentially trying 3 different versions of the data in this set.  One version contains all the features.  Another version has the standard features plus the calculated features of the home/away stats for a matchup.  The last version has the standard features plus the 'extended features' which not only includes the stats of the two teams in the match up but also the current teams previous opponents and the current teams opponents of the team opponents in that matchup.

In [ ]:
match_details = form_data.get_coverage()
pd.set_option('display.max_columns', 500)
display(match_details.head())

In [2]:
data_csv = 'round' + str(round_number) + '.csv'

if testing:
    raw_data = form_data.run_data()
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)

# Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

raw_X, raw_y = split_target(raw_data)

display(raw_data.head())

print('Data Loaded...')

ROUND 4 :: TEAM ID 21
ROUND 5 :: TEAM ID 21
ROUND 6 :: TEAM ID 21
ROUND 7 :: TEAM ID 21
ROUND 8 :: TEAM ID 21
ROUND 9 :: TEAM ID 21
ROUND 10 :: TEAM ID 21
ROUND 11 :: TEAM ID 21
ROUND 12 :: TEAM ID 21
ROUND 13 :: TEAM ID 21
ROUND 14 :: TEAM ID 21
ROUND 15 :: TEAM ID 21
ROUND 16 :: TEAM ID 21
ROUND 18 :: TEAM ID 21
ROUND 19 :: TEAM ID 21
ROUND 21 :: TEAM ID 21
ROUND 23 :: TEAM ID 21
ROUND 24 :: TEAM ID 21
ROUND 25 :: TEAM ID 21
ROUND 26 :: TEAM ID 21
ROUND 5 :: TEAM ID 22
ROUND 6 :: TEAM ID 22
ROUND 7 :: TEAM ID 22
ROUND 8 :: TEAM ID 22
ROUND 9 :: TEAM ID 22
ROUND 10 :: TEAM ID 22
ROUND 11 :: TEAM ID 22
ROUND 12 :: TEAM ID 22
ROUND 13 :: TEAM ID 22
ROUND 15 :: TEAM ID 22


/Users/senzari/.local/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


ROUND 16 :: TEAM ID 22
ROUND 18 :: TEAM ID 22
ROUND 19 :: TEAM ID 22
ROUND 21 :: TEAM ID 22
ROUND 22 :: TEAM ID 22
ROUND 23 :: TEAM ID 22
ROUND 24 :: TEAM ID 22
ROUND 25 :: TEAM ID 22
ROUND 26 :: TEAM ID 22
ROUND 5 :: TEAM ID 23
ROUND 6 :: TEAM ID 23
ROUND 7 :: TEAM ID 23
ROUND 8 :: TEAM ID 23
ROUND 9 :: TEAM ID 23
ROUND 10 :: TEAM ID 23
ROUND 11 :: TEAM ID 23
ROUND 12 :: TEAM ID 23
ROUND 13 :: TEAM ID 23
ROUND 14 :: TEAM ID 23
ROUND 15 :: TEAM ID 23
ROUND 16 :: TEAM ID 23
ROUND 17 :: TEAM ID 23
ROUND 18 :: TEAM ID 23
ROUND 19 :: TEAM ID 23
ROUND 21 :: TEAM ID 23
ROUND 22 :: TEAM ID 23
ROUND 23 :: TEAM ID 23
ROUND 24 :: TEAM ID 23
ROUND 25 :: TEAM ID 23
ROUND 26 :: TEAM ID 23
ROUND 4 :: TEAM ID 24
ROUND 6 :: TEAM ID 24
ROUND 7 :: TEAM ID 24
ROUND 8 :: TEAM ID 24
ROUND 9 :: TEAM ID 24
ROUND 10 :: TEAM ID 24
ROUND 11 :: TEAM ID 24
ROUND 12 :: TEAM ID 24
ROUND 13 :: TEAM ID 24
ROUND 14 :: TEAM ID 24
ROUND 15 :: TEAM ID 24
ROUND 16 :: TEAM ID 24
ROUND 17 :: TEAM ID 24
ROUND 18 :: TEAM ID 2

,match_id,team_id,team_name,opp_id,opp_name,scheduled,games_played,is_home,current_formation,avg_points,avg_goals_for,avg_goals_against,margin,goal_diff,win_percentage,sos,opp_is_home,opp_formation,opp_avg_points,opp_avg_goals,opp_margin,opp_goal_diff,opp_win_percentage,opp_sos,current_team_possession,current_team_attacks,current_team_dangerous_attacks,current_team_yellow_cards,current_team_corner_kicks,current_team_shots_on_target,current_team_shots_total,current_team_ball_safe,current_team_goal_attempts,current_team_saves,current_team_first_half_goals,current_team_sec_half_goals,current_team_goal_kicks,opp_team_possession,opp_team_attacks,opp_team_dangerous_attacks,opp_team_yellow_cards,opp_team_corner_kicks,opp_team_shots_on_target,opp_team_shots_total,opp_team_ball_safe,opp_team_goal_attempts,opp_team_saves,opp_team_first_half_goals,opp_team_sec_half_goals,opp_team_goal_kicks,goals,points
0,249,21,FC Dallas,33,DC United,2016-03-26 21:30:00,3,0,4-2-3-1,2.000000,1.333333,1.666667,-0.333333,-1,0.666667,0.404762,1,4-4-2,0.666667,0.666667,-1.000000,-3,0.357143,0.659892,50.333333,103.333333,60.666667,2.000000,6.666667,6.666667,11.666667,85.333333,11.666667,1.666667,0.333333,1.000000,5.666667,46.000000,97.666667,50.000000,2.666667,5.333333,3.666667,7.333333,94.333333,7.333333,1.333333,0.333333,0.333333,7.666667,3,3
1,255,21,FC Dallas,39,Columbus Crew,2016-04-03 01:00:00,3,1,4-4-2,2.000000,1.666667,1.666667,0.000000,0,0.714286,0.277778,0,4-2-3-1,0.333333,0.666667,-0.666667,-2,0.166667,0.616713,50.666667,112.000000,56.333333,2.333333,4.666667,5.666667,10.666667,94.666667,10.666667,1.666667,0.666667,1.000000,5.333333,57.000000,96.333333,64.666667,1.000000,7.333333,6.000000,12.333333,82.666667,13.000000,4.000000,0.000000,0.666667,5.000000,1,1
2,265,21,FC Dallas,32,San Jose Earthquakes,2016-04-10 01:00:00,3,1,4-4-2,2.333333,2.000000,0.333333,1.666667,5,0.833333,0.471751,0,4-4-2,1.333333,1.333333,-0.333333,-1,0.500000,0.674603,49.333333,113.000000,56.000000,2.000000,3.333333,6.333333,15.000000,95.333333,15.000000,3.000000,0.666667,1.333333,5.666667,48.333333,103.000000,50.333333,2.333333,3.666667,3.666667,6.333333,102.666667,6.333333,2.333333,0.666667,0.666667,10.333333,2,1
3,272,21,FC Dallas,30,Portland Timbers,2016-04-14 02:30:00,3,0,4-4-2,1.666667,2.000000,1.000000,1.000000,3,0.687500,0.461905,1,4-2-3-1,0.666667,1.333333,-1.000000,-3,0.357143,0.589444,51.333333,103.333333,48.333333,2.666667,5.000000,5.666667,13.666667,90.000000,13.666667,2.666667,1.000000,1.000000,5.333333,47.666667,100.000000,54.666667,2.666667,9.000000,3.666667,8.000000,91.000000,8.000000,3.000000,0.000000,1.333333,6.333333,3,3
4,282,21,FC Dallas,27,Sporting Kansas City,2016-04-17 23:00:00,3,1,4-4-2,1.666667,2.000000,1.000000,1.000000,3,0.687500,0.574074,0,4-3-3,2.000000,1.333333,0.666667,2,0.625000,0.589444,51.333333,103.333333,48.333333,2.666667,5.000000,5.666667,13.666667,90.000000,13.666667,2.666667,1.000000,1.000000,5.333333,47.666667,105.666667,50.666667,1.000000,3.333333,3.000000,7.333333,98.666667,7.333333,5.333333,0.333333,1.000000,10.666667,2,3


Data Loaded...


In [ ]:
display(raw_data)

In [ ]:
home_score = pd.crosstab(match_details['home_score'], 
            match_details.replace({'home_points': {0: 'Lose', 1: 'Tie', 3: 'Win'}})['home_points']).apply(lambda r: r/len(match_details), axis=1)
home_score['Total'] = home_score.sum(axis=1)
home_score.loc['Total']= home_score.sum()
display(home_score)

In [ ]:
away_score = pd.crosstab(match_details['away_score'], 
            match_details.replace({'away_points': {0: 'Lose', 1: 'Tie', 3: 'Win'}})['away_points']).apply(lambda r: r/len(match_details), axis=1)

away_score['Total'] = away_score.sum(axis=1)
away_score.loc['Total'] = away_score.sum()
display(away_score)

In [ ]:
away_shots = pd.crosstab(match_details['away_score'], 
            match_details['away_shots_total'])

away_shots['Total'] = away_shots.sum(axis=1)
away_shots.loc['Total'] = away_shots.sum()
display(away_shots)

In [ ]:
home_shots = pd.crosstab(match_details['home_score'], 
            match_details['home_shots_total'])

home_shots['Total'] = home_shots.sum(axis=1)
home_shots.loc['Total'] = home_shots.sum()
display(home_shots)

## Extended Features RELATIONSHIPS

In [ ]:
columns = ['e_f_shots_on_target', 'e_f_ball_safe', 'e_f_possession', 'e_f_attacks', 'e_f_dangerous_attacks', 'e_f_shots_total', 
           'points']
extended_features = raw_data[columns]
display(extended_features.head())

for c in range(len(columns)):
    if c < (len(columns)-1): 
        compare = (len(columns)-2) - c
        for x in (range(compare)):
            fig, ax = plt.subplots(figsize = (14,8))
            cmap = cm.get_cmap('gist_rainbow')

            for i, cluster in extended_features.groupby('points'):   
                cluster.plot(ax = ax, kind = 'scatter', x = columns[c], y = columns[x+1+c], 
                    color = cmap((i)*1.0/(4)), label = 'Points %i'%(i), s=30);


In [ ]:
#print(match_details)
cols = ['home_attacks', 'home_ball_safe', 'home_points']
rels = match_details[cols]
rels = rels.dropna()
display(rels.head())
print(rels.shape)

fig, ax = plt.subplots(figsize = (14,8))
cmap = cm.get_cmap('gist_rainbow')

for i, cluster in rels.groupby('home_points'):   
    cluster.plot(ax = ax, kind = 'scatter', x = 'home_attacks', y = 'home_ball_safe', 
                    color = cmap((i)*1.0/(4)), label = 'Points %i'%(i), s=30);

In [ ]:
#print(match_details)
cols = ['away_attacks', 'away_ball_safe', 'away_points']
rels = match_details[cols]
rels = rels.dropna()
display(rels.head())

fig, ax = plt.subplots(figsize = (14,8))
cmap = cm.get_cmap('gist_rainbow')

for i, cluster in rels.groupby('away_points'):   
    cluster.plot(ax = ax, kind = 'scatter', x = 'away_attacks', y = 'away_ball_safe', 
                    color = cmap((i)*1.0/(4)), label = 'Points %i'%(i), s=30);

In [ ]:
cols = ['home_shots_total', 'home_score', 'home_points']
rels = match_details[cols]
rels = rels.dropna()
# print(rels.groupby('home_shots_total').groups)
rels = rels.sort_values('home_shots_total')

a = pd.crosstab(rels['home_shots_total'], rels['home_score'])
a.plot(kind='bar', stacked=True)
a['Total'] = a.sum(axis=1)
a.loc['Total']= a.sum()
display(a)

b = pd.crosstab(rels['home_shots_total'], rels['home_score'])
b = b.loc[8:12]
b['Total'] = b.sum(axis=1)
b.loc['Total']= b.sum()
display(b)

In [ ]:
cols = ['home_shots_on_target', 'home_score', 'home_points']
rels = match_details[cols]
rels = rels.dropna()

rels = rels.sort_values('home_shots_on_target')

temp = pd.crosstab(rels['home_shots_on_target'], rels['home_score'])
temp.plot(kind='bar', stacked=True)

In [ ]:
cols = ['away_shots_total', 'away_score', 'away_points']
rels = match_details[cols]
rels = rels.dropna()

a = pd.crosstab(rels['away_shots_total'], rels['away_score'])
a.plot(kind='bar', stacked=True)
a['Total'] = a.sum(axis=1)
a.loc['Total']= a.sum()
display(a)


b = pd.crosstab(rels['away_shots_total'], rels['away_score'])
b = b.loc[6:10]
b['Total'] = b.sum(axis=1)
b.loc['Total']= b.sum()
display(b)

In [ ]:
cols = ['away_shots_on_target', 'away_score', 'away_points']
rels = match_details[cols]
rels = rels.dropna()

rels = rels.sort_values('away_shots_on_target')

temp = pd.crosstab(rels['away_shots_on_target'], rels['away_score'])
temp.plot(kind='bar', stacked=True)

In [ ]:
conv_points = match_details
def set_group(goals):
    if goals >= 2:
        return 1
    elif goals < 2:
        return 0
    
conv_points['home_converted_score'] = conv_points.apply(lambda row: set_group(row['home_score']), axis=1)
conv_points = conv_points.drop('home_score', 1)

conv_points['away_converted_score'] = conv_points.apply(lambda row: set_group(row['away_score']), axis=1)
conv_points = conv_points.drop('away_score', 1) 

In [ ]:
away_points = pd.crosstab(conv_points['away_converted_score'], 
            conv_points.replace({'away_points': {0: 'Lose', 1: 'Tie', 3: 'Win'}})['away_points']).apply(lambda r: r/len(match_details), axis=1)

away_points['Total'] = away_points.sum(axis=1)
away_points.loc['Total']= away_points.sum()
display(away_points)

In [ ]:
home_points = pd.crosstab(conv_points['home_converted_score'], 
            conv_points.replace({'home_points': {0: 'Lose', 1: 'Tie', 3: 'Win'}})['home_points']).apply(lambda r: r/len(match_details), axis=1)
home_points['Total'] = home_points.sum(axis=1)
home_points.loc['Total']= home_points.sum()
display(home_points)

In [ ]:
home_columns = ['current_team_home_possession', 'current_team_home_attacks', 'current_team_home_dangerous_attacks', 'current_team_home_shots_on_target', 'current_team_home_shots_total', 'current_team_home_ball_safe', 
           'points']

away_columns = ['current_team_away_possession', 'current_team_away_attacks', 'current_team_away_dangerous_attacks', 'current_team_away_shots_on_target', 'current_team_away_shots_total', 'current_team_away_ball_safe', 
           'points']
extended_features = raw_data[home_columns]
display(extended_features.head())

for c in range(len(home_columns)):
    if c < (len(home_columns)-1): 
        compare = (len(home_columns)-2) - c
        for x in (range(compare)):
            fig, ax = plt.subplots(figsize = (14,8))
            cmap = cm.get_cmap('gist_rainbow')

            for i, cluster in extended_features.groupby('points'):   
                cluster.plot(ax = ax, kind = 'scatter', x = home_columns[c], y = home_columns[x+1+c], 
                    color = cmap((i)*1.0/(4)), label = 'Points %i'%(i), s=30);


In [ ]:
home_columns = ['current_team_home_possession', 'current_team_home_attacks', 'current_team_home_dangerous_attacks', 'current_team_home_shots_on_target', 'current_team_home_shots_total', 'current_team_home_ball_safe', 
           'points']

away_columns = ['current_team_away_possession', 'current_team_away_attacks', 'current_team_away_dangerous_attacks', 'current_team_away_shots_on_target', 'current_team_away_shots_total', 'current_team_away_ball_safe', 
           'points']
extended_features = raw_data[away_columns]
display(extended_features.head())

for c in range(len(away_columns)):
    if c < (len(away_columns)-1): 
        compare = (len(away_columns)-2) - c
        for x in (range(compare)):
            fig, ax = plt.subplots(figsize = (14,8))
            cmap = cm.get_cmap('gist_rainbow')

            for i, cluster in extended_features.groupby('points'):   
                cluster.plot(ax = ax, kind = 'scatter', x = away_columns[c], y = away_columns[x+1+c], 
                    color = cmap((i)*1.0/(4)), label = 'Points %i'%(i), s=30);

In [ ]:
discovery_columns = ['current_formation', 'opp_formation', 'current_team_home_attacks', 'current_team_away_attacks', 'current_opp_away_attacks', 'current_team_home_shots_on_target', 'current_team_home_corner_kicks', 'current_team_away_corner_kicks', 
           'current_opp_home_attacks', 'avg_goals_for', 'avg_goals_against', 'points']

extended_features = raw_data[discovery_columns]

rf = model_libs.reformat_formation(extended_features, extended_features.groupby('current_formation').groups, extended_features.groupby('opp_formation').groups)

for r in range(len(rf)):
    extended_features = extended_features.replace(rf[r], r)

#display(extended_features.head())
#display(extended_features.tail())

for c in range(len(discovery_columns)):
    if c < (len(discovery_columns)-1): 
        compare = (len(discovery_columns)-2) - c
        for x in (range(compare)):
            fig, ax = plt.subplots(figsize = (14,8))
            cmap = cm.get_cmap('gist_rainbow')

            for i, cluster in extended_features.groupby('points'):
                cluster.plot(ax = ax, kind = 'scatter', x = discovery_columns[c], y = discovery_columns[x+1+c], 
                    color = cmap((i)*1.0/(4)), label = 'Points %i'%(i), s=30);